# 從網路抓資料

20161211

## 安裝

如果沒有root權限的安裝方法

In [1]:
!pip3 install -IU requests --user

    100% |████████████████████████████████| 583kB 576kB/s 


有root 權限的安裝方法

In [3]:
!sudo -H pip3 install -IU requests

Password:


## 下載資料
- 引入package - [requests](http://docs.python-requests.org/en/master/)

In [16]:
import requests

- 參考下載檔案的方法[連結](http://stackoverflow.com/questions/16694907/how-to-download-large-file-in-python-with-requests-py), 先設立一個函數，需要輸入要存放的檔案路徑folder_path與要下載的網址url

In [6]:
a = "https://survey.banking.gov.tw/statis/2_1_1050910509/22010.xls".split("/")
a

['https:',
 '',
 'survey.banking.gov.tw',
 'statis',
 '2_1_1050910509',
 '22010.xls']

In [12]:
a[3:]  # -3 <= x < -1

['statis', '2_1_1050910509', '22010.xls']

In [13]:
b = {}
b["22010.xls"] = "全台灣銀行存款餘額.xls"
b[a[-1]]

'全台灣銀行存款餘額.xls'

In [19]:
def download_file(folder_path,url):
    b = {}
    b["22010.xls"] = "全台灣銀行存款餘額.xls"
    # 給定下載檔案的存放路徑+名稱
    local_filename = folder_path+b[url.split('/')[-1]]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as aha:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                aha.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return local_filename

 - 去[銀行局網站](https://survey.banking.gov.tw/statis/stmain.jsp?sys=100&funid=r100),我們優先看**2-1 一般銀行及信用合作社存款月底餘額**這個連結

在windows 下請用

In [ ]:
!dir/w

在 Linux / MacOS

In [2]:
!pwd

/Users/aha/Desktop/Python


- ./ 當前目錄
- ../ 上一層目錄

In [21]:
download_file('./',"https://survey.banking.gov.tw/statis/2_1_1050910509/22010.xls")

'./全台灣銀行存款餘額.xls'

- 確認檔案是否存在

In [14]:
!ls -alh |grep '22010.xls'

-rw-r--r--   1 aha  staff    81K Dec 11 00:32 22010.xls


- 因為是 xls 結尾, 這表示這個檔案是 excel 2003版本，需要用到xlrd這個package 才能讀取

In [16]:
# 小撇步, 如果是mac
!open ./22010.xls

### 讀取檔案

可以參考我的[github](https://github.com/ntuaha/TWFS),這次處理的檔案，可以用這個[範例程式](https://github.com/ntuaha/TWFS/blob/master/src/transform/transform_MD_Bal.py)

In [22]:
import xlrd
book = xlrd.open_workbook("./22010.xls")
sh = book.sheet_by_index(0)

來讀取第一個欄位看看

In [27]:
sh.cell_value(rowx=8,colx = 0).replace("\u3000","")

'總計'

In [23]:
sh.cell_value(rowx=0,colx = 0)

'2-1\u3000一般銀行及信用合作社存款月底餘額'

把怪異的 \u3000 處理掉，最好的方法就是代換掉他

In [25]:
sh.cell_value(rowx=0,colx = 0).replace("\u3000"," ")

'2-1 一般銀行及信用合作社存款月底餘額'

In [26]:
sh.cell_value(rowx=9,colx = 0).replace("\u3000"," ")

'Total'

In [30]:
sh.cell_value(rowx=10,colx = 0).replace("\u3000"," ")

'臺灣銀行       #'

## 作業
請設計一個python檔案，可以自動化把這些資料讀取下來後，整理成一個漂亮個格式，然後用excel 2010以上的版本輸出．

In [61]:

a = requests.get("https://www.esunbank.com.tw/bank/personal/deposit/rate/forex/foreign-exchange-rates")

In [64]:
cookies = a.cookies.get_dict()
headers = {'Content-Type': 'application/json; charset=UTF-8','User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36"}
headers["X-Requested-With"] = "XMLHttpRequest"

b = requests.post("https://www.esunbank.com.tw/bank/Layouts/esunbank/Deposit/DpService.aspx/GetRateQuoteTime",data="{day:'2016-12-09',time:'22:59:45',rateType:'ASLL'}",cookies = cookies,headers=headers)
cookies = b.cookies.get_dict()
b.text

'{"d":""}'

In [59]:
import json
b = requests.post("https://www.esunbank.com.tw/bank/Layouts/esunbank/Deposit/DpService.aspx/GetForeignExchageRate",data="{day:'2016-12-09',time:'22:59:45'}",cookies = cookies,headers=headers)

In [51]:
b.text

'{"d":""}'